In [ ]:
using DelimitedFiles,HDF5, PyCall, PyPlot, NRRD,FileIO, TiffImages,Images, ProgressMeter, Statistics,LinearAlgebra, NaNStatistics
using _Data

In [ ]:
@pyimport numpy
@pyimport matplotlib.patches as patches
@pyimport scipy.ndimage as ndimage
@pyimport pandas as pd
@pyimport seaborn as sns
@pyimport scipy.stats as stats

In [ ]:
include("../../../functions/func_map.jl")
include("../../../functions/func_stat.jl")
include("../../../functions/func_data.jl")
include("../../../functions/func_plot.jl")

In [ ]:
rc_params = PyDict(pyimport("matplotlib")["rcParams"]);
rc_params["font.sans-serif"] = ["Arial"];
rc_params["font.size"] = 7;
rc_params["lines.linewidth"] = 1;
rc_params["lines.markersize"] = 4;
rc_params["xtick.major.size"] = 2;
rc_params["ytick.major.size"] = 2;
rc_params["xtick.major.pad"] = 2;
rc_params["ytick.major.pad"] = 2;
# rc_params["axes.labelpad"] = 2;

cim(img::Matrix{UInt32}) = CairoImageSurface(img, Cairo.FORMAT_RGB24; flipxy = false) 
cim(img::Matrix{ARGB32}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{RGB24}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{UInt8}) = cim(Gray.(reinterpret(N0f8, img)))
cim(img::Array{UInt8,3}) = cim(RGB24.(reinterpret(N0f8, img[:,:,1]), reinterpret(N0f8, img[:,:,2]), reinterpret(N0f8, img[:,:,3])));downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);
downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);

In [ ]:
fig_path = "/home/chuyu/Notebooks/project_place_cell/figures/output/figure1"
mkpath(fig_path)

In [ ]:
ds_save_cy_ref = Dataset("20220416_123633", "chuyu", gethostname() == "roli-$(5)" ? "/data" : "/nfs/data$(5)")
# load atlas 
atlas_ref_img = TiffImages.load(data_path(ds_save_cy_ref)*"/Elavl3-H2BRFP.tif");
atlas_ref = convert(Array{Float32}, atlas_ref_img);

stack_img = fill(NaN32, size(atlas_ref,2), size(atlas_ref,1), size(atlas_ref,3))
for (i, z) in enumerate(1:size(atlas_ref,3))
    stack_img[:,:,i].= atlas_ref[:,:,z]'
end




In [ ]:
data_all_exp = 
[
    ["20220407_152537", 4, "jen"],
    ["20220406_111526", 9, "jen"],
    ["20220407_090156", 5, "jen"],
    ["20220417_165530", 2, "jen"],
    ["20220406_153842", 9, "jen"],
    ["20220405_171444", 25, "jen"],
    ["20220416_160516", 6, "jen"]
];

# Anatomy

In [ ]:
distance_all = zeros(Float32, 11,11,11)
x_index, y_index, z_index = numpy.meshgrid(1:size(distance_all, 1),1:size(distance_all, 2), 1:size(distance_all, 3))
for i in 1:length(x_index)
    x = x_index[i]
    y = y_index[i]
    z = z_index[i]
    distance_all[x, y, z] = norm([x-Int32.(ceil(size(distance_all, 1)/2)), y-Int32.(ceil(size(distance_all, 2)/2)), 2*(z-Int32.(ceil(size(distance_all, 3)/2)))]);
end

kernel = Int32.(collect(distance_all.<=5)); #check this

In [ ]:
@pyimport matplotlib.colors as mpl_colors
@pyimport matplotlib.cm as cm 

In [ ]:
hot = cm.get_cmap("hot", 100)
alpha_top = 50
cmaplist = [hot(i) for i in 1:(hot.N)]
cmaplist_new = []
length_cmaplist = length(cmaplist)
for (i, color_) in enumerate(cmaplist)
    new_color_ = numpy.copy(color_)
    if i<=alpha_top
        new_color_[4] = (100/alpha_top)*i/length_cmaplist
    end
    append!(cmaplist_new,[new_color_]) 
end
hot_new = mpl_colors.LinearSegmentedColormap.from_list("hot_new",cmaplist_new,100)

In [ ]:
## cell loc
analyzer = "chuyu"
cell_locs_all = []
cell_locs_all_original = []
experiment_filename_all = []
specificity_population_z_all = []
place_cell_index_all = []
for which_combo in 1:length(data_all_exp)
    data_combo = data_all_exp[which_combo]
    experiment_filename_1 = data_combo[1]
    server_1 = data_combo[2]

    experimenter = data_combo[end]
    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")

    NMF_filename = joinpath(data_path(ds_save_cy_1), "NMF_merge.h5")
    NMF_file = h5open(NMF_filename, "r")
    global Z_all = HDF5.readmmap(NMF_file["Z_all"])
    global X_all = HDF5.readmmap(NMF_file["X_all"])
    global Y_all = HDF5.readmmap(NMF_file["Y_all"])
    global neuron_label = HDF5.readmmap(NMF_file["neuron_label"])
    close(NMF_file)

    n_neurons = length(X_all)
    cell_locs = fill(NaN32, n_neurons, 3)
    cell_locs[:, 1] = X_all
    cell_locs[:, 2] = Y_all
    cell_locs[:, 3] = Z_all
    append!(cell_locs_all_original,[cell_locs]);
    
    all_files = readdir(data_path(ds_save_cy_1))
    long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
    spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
    candidate_filename = long_name_files[spatial_info_index]
    which_file = [occursin(experiment_filename_1, candidate_filename[i])*!occursin("shuffleall", candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
    @assert(length(candidate_filename[which_file]) == 1)
    save_file_name = candidate_filename[which_file][1]
    info_filename = joinpath(data_path(ds_save_cy_1), save_file_name)
    file = h5open(info_filename, "r")
    specificity_population_z_1 = HDF5.readmmap(file["specificity_population_z"])
    if haskey(file, "place_cell_index")
        place_cell_index = HDF5.readmmap(file["place_cell_index"])    
    else
        place_cell_index = read(h5open(joinpath(data_path(ds_save_cy_1), "compare_map_results_original.h5")), "place_cell_index_1")
    end    
    close(file)
    
    append!(specificity_population_z_all,[specificity_population_z_1])
        
    loc_save_path = joinpath(data_path(ds_save_cy_1), "$experiment_filename_1"*"_engert")
    registered_cell_locs_original = readdlm(joinpath(loc_save_path, experiment_filename_1*"_nmf_loc_engert_registered.txt"));
    registered_cell_locs = registered_cell_locs_original[:,1:3];


    atlas_cell_locs = Int32.(round.(registered_cell_locs));
    atlas_cell_locs[:,1] .+= 329;
    atlas_cell_locs[:,2] .+= 41;

    atlas_cell_locs[:,1] .= size(stack_img,1).-atlas_cell_locs[:,1];
    atlas_cell_locs[:,3] .= size(stack_img,3).-atlas_cell_locs[:,3];

    cell_locs = fill(NaN32, n_neurons, 3)
    @showprogress for (i, which_neuron) in enumerate(1:n_neurons)
        cell_locs[i, :] = mean(atlas_cell_locs[neuron_label.==which_neuron, :], dims=1)

    end
    
    append!(cell_locs_all,[cell_locs]);
    append!(experiment_filename_all,[experiment_filename_1]);
    append!(place_cell_index_all, [place_cell_index])
end

In [ ]:
cell_number_all = []
for which_combo in 1:length(data_all_exp)
    data_combo = data_all_exp[which_combo]
    experiment_filename_1 = data_combo[1]
    server_1 = data_combo[2]

    experimenter = data_combo[end]
    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")

    NMF_filename = joinpath(data_path(ds_save_cy_1), "NMF_merge.h5")
    NMF_file = h5open(NMF_filename, "r")
    global Z_all = HDF5.readmmap(NMF_file["Z_all"])
    global X_all = HDF5.readmmap(NMF_file["X_all"])
    global Y_all = HDF5.readmmap(NMF_file["Y_all"])
    global neuron_label = HDF5.readmmap(NMF_file["neuron_label"])
    close(NMF_file)
    
    println(nanmaximum(neuron_label))
    append!(cell_number_all, nanmaximum(neuron_label))

    
end

In [ ]:
# exluding one fish solved with eta0

mean([

86117.0
71361.0
78946.0
64333.0
81924.0
59044.0
])

In [ ]:
std([

86117.0
71361.0
78946.0
64333.0
81924.0
59044.0
])

In [ ]:
for which_combo in 1:length(data_all_exp)
    data_combo = data_all_exp[which_combo]
    experiment_filename_1 = data_combo[1]
    server_1 = data_combo[2]

    experimenter = data_combo[end]
    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")

    NMF_filename = joinpath(data_path(ds_save_cy_1), "NMF_merge.h5")
    NMF_file = h5open(NMF_filename, "r")
    global Z_all = HDF5.readmmap(NMF_file["Z_all"])
    global X_all = HDF5.readmmap(NMF_file["X_all"])
    global Y_all = HDF5.readmmap(NMF_file["Y_all"])
    global neuron_label = HDF5.readmmap(NMF_file["neuron_label"])
    close(NMF_file)
    
    println(1- nanmaximum(neuron_label)./sum(isfinite.(neuron_label)))
end

In [ ]:
mean([0.2678433
0.27929103
0.28596365
0.3102498
0.31566328
0.24656099])

In [ ]:
std([0.2678433
0.27929103
0.28596365
0.3102498
0.31566328
0.24656099])

In [ ]:
for place_cell_index in place_cell_index_all
    println(length(place_cell_index))
end

In [ ]:
mean([length(place_cell_index_all[i]) for i in 1:length(place_cell_index_all)])

In [ ]:
std([length(place_cell_index_all[i]) for i in 1:length(place_cell_index_all)])

In [ ]:
function fill_nan(vec, desired_length)
    original_length = length(vec)
    new_vec = fill!(Array{Float64,1}(undef, desired_length), NaN)
    new_vec[1:original_length] = vec
    return new_vec
end

In [ ]:
cell_locs_tog = reduce(vcat,cell_locs_all)[:,1:3];

In [ ]:
place_cell_locs_all = [cell_locs_all[i][place_cell_index_all[i],:] for i in 1:length(cell_locs_all)]
place_cell_locs_tog = reduce(vcat,place_cell_locs_all)[:,1:3];

In [ ]:
fig, ax = subplots(1,1,dpi=250, figsize=(15,10))
for (i,cell_locs) in enumerate(cell_locs_all)
    place_cell_index = Int32.(place_cell_index_all[i])
    ax.scatter(cell_locs[place_cell_index,1], cell_locs[place_cell_index,2], s=2, label=experiment_filename_all[i])
end
ax.set_aspect("equal", "box")
ax.legend()


fig_bg = maximum(stack_img, dims=3)[:,:,1]
ax.imshow(fig_bg', cmap="gray")
# axvline(620)
# axvline(450)

In [ ]:
for (i,cell_locs) in enumerate(cell_locs_all)
    fig, ax = subplots(1,1,dpi=250)

    place_cell_index = Int32.(place_cell_index_all[i])
    ax.scatter(cell_locs[place_cell_index,1], cell_locs[place_cell_index,2], s=2, label=experiment_filename_all[i])
    
    fig_bg = maximum(stack_img, dims=3)[:,:,1]
    ax.imshow(fig_bg', cmap="gray")
    
    ax.set_aspect("equal", "box")
    ax.legend()
    
    fig, ax = subplots(1,1,dpi=250)

    place_cell_index = Int32.(place_cell_index_all[i])
    ax.scatter(cell_locs[place_cell_index,1], cell_locs[place_cell_index,3], s=2, label=experiment_filename_all[i])
    
    fig_bg = maximum(stack_img, dims=2)[:,1,:]
    ax.imshow(fig_bg', cmap="gray",aspect=2/0.798)
    
    ax.legend()
end



In [ ]:
spatial_p_img = zeros(Int32, size(stack_img));
for i in 1:size(place_cell_locs_tog,1)
    x = Int32.(numpy.round(place_cell_locs_tog[i,1]))
    y = Int32.(numpy.round(place_cell_locs_tog[i,2]))
    z = Int32.(numpy.round(place_cell_locs_tog[i,3]))
    if x.<=size(spatial_p_img,1) && y.<=size(spatial_p_img,2) && z.<=size(spatial_p_img,3)
        spatial_p_img[x,y,z] += 1
    end
end

In [ ]:
fig_bg = maximum(spatial_p_img, dims=3)[:,:,1]
fig, ax = subplots(1,1,dpi=250)
ax.imshow(fig_bg', cmap="gray")

In [ ]:
spatial_p_img_convolved = ndimage.convolve(spatial_p_img, kernel);

In [ ]:
extrema(reduce(vcat, cell_locs_all)[:,1])

In [ ]:
extrema(reduce(vcat, cell_locs_all)[:,2])

In [ ]:
extrema(reduce(vcat, cell_locs_all)[:,3])

In [ ]:
hist(reduce(vcat, cell_locs_all)[:,3])

In [ ]:
stack_img_cropped = copy(stack_img)



stack_img_cropped[1:98,:,:] .= NaN32;
stack_img_cropped[1090:end,:,:] .= NaN32;

stack_img_cropped[:,1:10,:] .= NaN32;
stack_img_cropped[:,615:end,:] .= NaN32;

stack_img_cropped[:,:,1:25] .= NaN32;
stack_img_cropped[:,:,129:end] .= NaN32;


In [ ]:
spatial_p_img_copy = Float32.(spatial_p_img_convolved)

spatial_p_img_copy[isnan.(stack_img_cropped)] .= NaN32
# spatial_p_img_copy[spatial_p_img_copy.<3] .= NaN
vmax=15

fig, ax = subplots(1,1,dpi=250)
fig_bg = nanmaximum(stack_img_cropped, dims=3)[:,:,1]
spatial_p_bg = numpy.nanmax(spatial_p_img_copy, axis=2)
ax.imshow(fig_bg', cmap="gray")
sp1 =ax.imshow(spatial_p_bg', cmap=hot_new, vmin=0, vmax=vmax)
# colorbar(sp1)
axis("off")
fig.savefig(joinpath(fig_path, "distribution_anatomy_1.png"), bbox_inches = "tight",transparent = true,pad_inches = 0)
fig.savefig(joinpath(fig_path, "distribution_anatomy_1.pdf"), bbox_inches = "tight",transparent = true,pad_inches = 0)

fig, ax = subplots(1,1,dpi=250)
fig_bg = nanmaximum(stack_img_cropped, dims=2)[:,1,:]
spatial_p_bg = numpy.nanmax(spatial_p_img_copy, axis=1)
ax.imshow(fig_bg', cmap="gray",aspect=2/0.798)
sp2 =ax.imshow(spatial_p_bg', cmap=hot_new,aspect=2/0.798, vmin=0, vmax=vmax)
axis("off")
fig.savefig(joinpath(fig_path, "distribution_anatomy_2.png"), bbox_inches = "tight",transparent = true,pad_inches = 0)
fig.savefig(joinpath(fig_path, "distribution_anatomy_2.pdf"), bbox_inches = "tight",transparent = true,pad_inches = 0)



fig, ax = subplots(1,1,dpi=250)
fig_bg = nanmaximum(stack_img_cropped, dims=1)[1,:,:]
spatial_p_bg = numpy.nanmax(spatial_p_img_copy, axis=0)
ax.imshow(fig_bg', cmap="gray",aspect=2/0.798)
sp3 =ax.imshow(spatial_p_bg', cmap=hot_new,aspect=2/0.798, vmin=0, vmax=vmax)
axis("off")
fig.savefig(joinpath(fig_path, "distribution_anatomy_3.png"), bbox_inches = "tight",transparent = true,pad_inches = 0)
fig.savefig(joinpath(fig_path, "distribution_anatomy_3.pdf"), bbox_inches = "tight",transparent = true,pad_inches = 0)

In [ ]:
fig = figure(figsize=(2,2))
sp1 =imshow(spatial_p_bg, cmap=hot_new, vmin=0, vmax=vmax)

sp1.set_visible(false)
axis("off")
clb = colorbar(shrink=0.3, aspect=8, mappable=sp1, ticks=[0, vmax], fraction=0.999, pad=0)
clb.ax.set_facecolor((0.2, 0.2, 0.2, 1.0))
clb.ax.tick_params(colors="white", width=1,length=0) 
clb.outline.set_color("white")
clb.outline.set_linewidth(1)
fig.savefig(joinpath(fig_path, "distribution_anatomy_colorbar.pdf"), bbox_inches = "tight",transparent = true,pad_inches = 0)

In [ ]:
@pyimport matplotlib.animation as anim

spatial_p_img_copy = Float32.(spatial_p_img_convolved)


fig = figure(figsize=(5,5), dpi=150)
function make_frame(i)
    clf()
    stack_img_z = stack_img[:,:,i]
    img_copy_z = spatial_p_img_copy[:,:,i]
    imshow(stack_img_z', cmap="gray", vmin=0, vmax=0.6)
    imshow(img_copy_z', cmap=hot_new, vmin=0, vmax=15)
    which_z = size(spatial_p_img_copy, 3) - i +1
    title("z = $which_z")
    axis("off")
    display(gcf())
    IJulia.clear_output(true)
    # sleep(0.2)
    # close(fig)
end

withfig(fig) do
    myanim = anim.FuncAnimation(fig, make_frame, frames=reverse(1:size(spatial_p_img_copy, 3)), interval=100)
    myanim[:save]("place_cell_loc.mp4", bitrate=-1, extra_args=["-vcodec", "libx264", "-pix_fmt", "yuv420p"])
end

# Statistics

In [ ]:
function neuron_region(region_roi_bool, region_name, neuron_label)
    n_neuron = Int32(nanmaximum(neuron_label))
    # for one merged cell, it belongs to telecephalon if at least one of its roi belongs to telencephalon
    region_roi_bool = region_roi_bool[:,findall(region_names .== region_name)][:,1]
    whether_region = falses(n_neuron)
    for which_neuron in Int32.(numpy.unique(neuron_label)[1:end-1])
        if sum(region_roi_bool[neuron_label.==which_neuron]) >0
            whether_region[which_neuron] = true
        end
    end
    return whether_region
end

In [ ]:
which_regions = ["Telencephalon -", "Diencephalon -", "Mesencephalon -", "Rhombencephalon -", "Telencephalon - Pallium", "Telencephalon - Subpallium"]

In [ ]:
region_percentage = []
region_cell_number = []
region_place_cell_number = []

@showprogress for which_data in 1:length(data_all_exp)
    
    place_cell_index = place_cell_index_all[which_data]
    
    data_info = data_all_exp[which_data]

    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]
    experimenter = data_info[end]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")


    NMF_filename = joinpath(data_path(ds_save_cy_1), "NMF_merge.h5")
    NMF_file = h5open(NMF_filename, "r")
    global neuron_label = HDF5.readmmap(NMF_file["neuron_label"])
    close(NMF_file)


    # whether individual roi belongs to a certain region
    region_bool_filename = joinpath(data_path(ds_save_cy_1), "region_roi_bool.h5")
    region_bool_file = h5open(region_bool_filename, "r")
    global region_names = read(region_bool_file, "region_names")
    global region_roi_bool = read(region_bool_file, "region_roi_bool")
    close(region_bool_file)

    region_percentage_this_fish = Dict()
    region_cell_number_this_fish = Dict()
    region_place_cell_number_this_fish = Dict()
    

    for region_name in which_regions
        whether_region = neuron_region(region_roi_bool, region_name, neuron_label)
        region_neurons = findall(whether_region)
        region_place_cell = findall(whether_in(region_neurons, place_cell_index))
        region_percentage_this_fish[region_name] = length(region_place_cell)/length(region_neurons)
        
        region_cell_number_this_fish[region_name] = length(region_neurons)
        region_place_cell_number_this_fish[region_name] = length(region_place_cell)
        
    end


    append!(region_percentage, [region_percentage_this_fish])
    append!(region_cell_number, [region_cell_number_this_fish])
    append!(region_place_cell_number, [region_place_cell_number_this_fish])
end

In [ ]:
tel_fraction = []
for i = 1:length(region_cell_number)
    tel_number = region_cell_number[i]["Telencephalon -"]
    cell_number = cell_number_all[i]
    

    println(tel_number)
    println(cell_number)
    append!(tel_fraction, tel_number/cell_number)
    # append!(tel_fraction, tel_number/sum([region_cell_number[i][key] for key in keys(region_cell_number[i])]))

end



In [ ]:
mean(tel_fraction)*100

In [ ]:
std(tel_fraction)*100

In [ ]:
region_place_cell_number

In [ ]:
interested_large_regions =  ["Telencephalon -", "Diencephalon -", "Mesencephalon -", "Rhombencephalon -"]

interested_small_regions =  ["Telencephalon - Pallium", "Telencephalon - Subpallium"]

In [ ]:
df = pd.DataFrame(region_place_cell_number)
# sns.set_theme(style="whitegrid")
# tips = sns.load_dataset("tips")
fig, ax = subplots(1,1,figsize=(1,2))
sns.stripplot(order = interested_large_regions, data=df,ax=ax, color=hot_new(60), s=3, alpha=0.8)

ax.set_ylabel("Place cell number",labelpad=-10)
ax.spines["top"].set_visible(false)
ax.spines["right"].set_visible(false)
ax.set_ylim(-20,1500)



order = interested_large_regions
for i in 0:length(order)-1
    m = numpy.nanmedian([x[order[i+1]] for x in region_place_cell_number])
    ax.plot([i-0.3, i+0.3], [m, m], c="k", lw=2)
end

xticks([0,1,2,3], labels= ["Tel.", "Di.", "Mes.", "Rhomb."], rotation=45, ha="right" , rotation_mode="anchor")
yticks([0,500,1000,1500],[0,"","",1500])

# fig.savefig(joinpath(fig_path, "region_number_large.png"), bbox_inches = "tight",transparent = true,pad_inches = 0)
fig.savefig(joinpath(fig_path, "region_number_large.pdf"), bbox_inches = "tight",transparent = true,pad_inches = 0)

In [ ]:
df = pd.DataFrame(region_place_cell_number)
# sns.set_theme(style="whitegrid")
# tips = sns.load_dataset("tips")
fig, ax = subplots(1,1,figsize=(0.5,2))
sns.stripplot(order = interested_small_regions, data=df,ax=ax, color=hot_new(60), s=3, alpha=0.8)
xticks([0,1], labels= ["Pallium", "Dorsal subpallium"], rotation=45, ha="right" , rotation_mode="anchor")
# ax.set_ylabel("Percentage")
ax.spines["top"].set_visible(false)
ax.spines["right"].set_visible(false)
ax.set_ylim(-20,1500)
yticks([])



order = interested_small_regions
for i in 0:length(order)-1
    m = numpy.nanmedian([x[order[i+1]] for x in region_place_cell_number])
    ax.plot([i-0.3, i+0.3], [m, m], c="k", lw=2)
end



# fig.savefig(joinpath(fig_path, "region_number_small.png"), bbox_inches = "tight",transparent = true,pad_inches = 0)
fig.savefig(joinpath(fig_path, "region_number_small.pdf"), bbox_inches = "tight",transparent = true,pad_inches = 0)

# PC specificity comparision

In [ ]:
specificity_fore_brain_all = []
specificity_hind_brain_all = []
specificity_whole_brain_all = []

spatial_info_fore_brain_all = []
spatial_info_hind_brain_all = []
spatial_info_whole_brain_all = []
@showprogress for which_data in 1:length(data_all_exp)
    
    place_cell_index = place_cell_index_all[which_data]
    
    data_info = data_all_exp[which_data]

    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]
    experimenter = data_info[end]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")


    NMF_filename = joinpath(data_path(ds_save_cy_1), "NMF_merge.h5")
    NMF_file = h5open(NMF_filename, "r")
    global neuron_label = HDF5.readmmap(NMF_file["neuron_label"])
    close(NMF_file)


    # whether individual roi belongs to a certain region
    region_bool_filename = joinpath(data_path(ds_save_cy_1), "region_roi_bool.h5")
    region_bool_file = h5open(region_bool_filename, "r")
    global region_names = read(region_bool_file, "region_names")
    global region_roi_bool = read(region_bool_file, "region_roi_bool")
    close(region_bool_file)
    
    all_files = readdir(data_path(ds_save_cy_1))
    long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
    spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
    candidate_filename = long_name_files[spatial_info_index]
    which_file = [occursin(experiment_filename_1, candidate_filename[i])*!occursin("shuffleall", candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
    @assert(length(candidate_filename[which_file]) == 1)
    save_file_name = candidate_filename[which_file][1]
    info_filename = joinpath(data_path(ds_save_cy_1), save_file_name)
    file = h5open(info_filename, "r")
    specificity = HDF5.readmmap(file["specificity"])
    spatial_info_all = HDF5.readmmap(file["spatial_info_all"])

    close(file)

    region_percentage_this_fish = Dict()
    region_cell_number_this_fish = Dict()
    region_place_cell_number_this_fish = Dict()
    

    for region_name in ["Mesencephalon -", "Rhombencephalon -"]
        whether_region = neuron_region(region_roi_bool, region_name, neuron_label)
        region_neurons = findall(whether_region)
        region_place_cell = region_neurons[findall(whether_in(region_neurons, place_cell_index))]
        
        append!(specificity_hind_brain_all, [specificity[region_place_cell]])
        append!(spatial_info_hind_brain_all, [spatial_info_all[region_place_cell]])
 
    end


   for region_name in ["Telencephalon -"]
        whether_region = neuron_region(region_roi_bool, region_name, neuron_label)
        region_neurons = findall(whether_region)
        region_place_cell = region_neurons[findall(whether_in(region_neurons, place_cell_index))]
        
        append!(specificity_fore_brain_all, [specificity[region_place_cell]])
        append!(spatial_info_fore_brain_all, [spatial_info_all[region_place_cell]])
    end
    
    append!(specificity_whole_brain_all, [spatial_info_all[place_cell_index]])
    append!(spatial_info_whole_brain_all, [spatial_info_all[place_cell_index]])

end

In [ ]:
function fill_nan(vec, desired_length)
    original_length = length(vec)
    new_vec = fill!(Array{Float64,1}(undef, desired_length), NaN)
    new_vec[1:original_length] = vec
    return new_vec
end

In [ ]:
specificity_data = Dict()
specificity_data["Forebrain"] = reduce(vcat, specificity_fore_brain_all)
specificity_data["Mid-hindbrain"] = reduce(vcat, specificity_hind_brain_all)
desired_length = maximum(length(specificity_data[key]) for key in keys(specificity_data))
for key in keys(specificity_data)
    specificity_data[key] = fill_nan(specificity_data[key], desired_length)
end


In [ ]:
stats.mannwhitneyu(reduce(vcat, specificity_fore_brain_all), reduce(vcat, specificity_hind_brain_all), alternative = "greater")

In [ ]:
order = ["Forebrain","Mid-hindbrain"]
bins= numpy.linspace(0,1,80)

tab10 = cm.get_cmap("tab10", 10)
color1 = tab10(0)
color2 = tab10(1)

fig, ax = subplots(1,1,figsize=(1,1.5))
hist(specificity_data["Forebrain"], label="Tel.",bins=bins, histtype="stepfilled", fc=(color1[1], color1[2], color1[3], 0.5),ec=color1,density=true, zorder= 1)
hist(specificity_data["Mid-hindbrain"], label="Mes. + Rhomb.",bins=bins, histtype="stepfilled", fc=(color2[1], color2[2], color2[3], 0.5),ec=color2,density=true, zorder= 0)
ax.legend(frameon=false,bbox_to_anchor=(-0.3, 1.02, 1, 0.2), loc="lower left",
                mode="expand", borderaxespad=0, handlelength=1.3)

xlabel("Place cell specificity\n(bits)", labelpad=5)
ylabel("Density", labelpad=-5)
ax.spines["top"].set_visible(false)
ax.spines["right"].set_visible(false)
xlim(0,0.63)
yticks([0,5,10,15],[0,"","","15"])

ylim(-0.2, 15)

fig.savefig(joinpath(fig_path, "fore_hind_specificity_significance.pdf"), bbox_inches = "tight")

In [ ]:
fig_path

In [ ]:
df = pd.DataFrame.from_dict(specificity_data)
df

In [ ]:
@pyimport os
@pyimport openpyxl

save_path = "../../../figures/source_data/Source Data Fig. 1.xlsx"
sheet_name = "Fig. 1h"

exists = os.path.isfile(save_path)
if exists
    writer = pd.ExcelWriter(save_path, engine = "openpyxl", mode="a", if_sheet_exists="replace")
    df.to_excel(writer, sheet_name = sheet_name)
    writer.close()
else
    df.to_excel(save_path, sheet_name)  
end

In [ ]:
specificity_data = Dict()
specificity_data["Forebrain"] = reduce(vcat, spatial_info_fore_brain_all)
specificity_data["Mid-hindbrain"] = reduce(vcat, spatial_info_hind_brain_all)
specificity_data["Whole brain"] = reduce(vcat, spatial_info_whole_brain_all)
desired_length = maximum(length(specificity_data[key]) for key in keys(specificity_data))
for key in keys(specificity_data)
    specificity_data[key] = fill_nan(specificity_data[key], desired_length)
end


In [ ]:
fig_path = "/home/chuyu/Notebooks/project_place_cell/figures/output/sfigure3"

In [ ]:
stats.mannwhitneyu(reduce(vcat, spatial_info_fore_brain_all), reduce(vcat, spatial_info_hind_brain_all), alternative = "greater")

In [ ]:
order = ["Forebrain","Mid-hindbrain", "Whole brain"]
bins= numpy.linspace(0,70,80)

tab10 = cm.get_cmap("tab10", 10)
color1 = tab10(0)
color2 = tab10(1)
color3 = tab10(2)

fig, ax = subplots(1,1,figsize=(1,1.5))
hist(specificity_data["Forebrain"], label="Tel.",bins=bins, histtype="stepfilled", fc=(color1[1], color1[2], color1[3], 0.5),ec=color1,density=true, zorder= 1)
hist(specificity_data["Mid-hindbrain"], label="Mes. + Rhomb.",bins=bins, histtype="stepfilled", fc=(color2[1], color2[2], color2[3], 0.5),ec=color2,density=true, zorder= 0)

ax.legend(frameon=false,bbox_to_anchor=(-0.3, 1.02, 1, 0.2), loc="lower left",
                mode="expand", borderaxespad=0, handlelength=1.3)
xlabel("Place cell SI\n(∝ bits/s)", labelpad=5, family=["Arial", "DejaVu Sans"])
ylabel("Density", labelpad=-5)
ax.spines["top"].set_visible(false)
ax.spines["right"].set_visible(false)
xlim(0,62)
yticks([0,0.1, 0.2],[0,"","0.2"])

ylim(-0.01, 0.27)
fig.savefig(joinpath(fig_path, "fore_hind_spatialinfo_significance.pdf"), bbox_inches = "tight")

In [ ]:
order = ["Forebrain","Mid-hindbrain", "Whole brain"]
bins= numpy.linspace(0,70,80)

tab10 = cm.get_cmap("tab10", 10)
color1 = tab10(0)
color2 = tab10(1)
color3 = tab10(2)

fig, ax = subplots(1,1,figsize=(1,1.5))
hist(specificity_data["Whole brain"], label="Whole brain",bins=bins, histtype="stepfilled", fc=(color3[1], color3[2], color3[3], 0.5),ec=color3,density=true, zorder= 0)
ax.legend(frameon=false,bbox_to_anchor=(-0.3, 1.02, 1, 0.2), loc="lower left",
                mode="expand", borderaxespad=0, handlelength=1.3)

xlabel("Place cell SI\n(∝ bits/s)", labelpad=5, family=["Arial", "DejaVu Sans"])
ylabel("Density", labelpad=-5)
ax.spines["top"].set_visible(false)
ax.spines["right"].set_visible(false)
xlim(0,62)
yticks([0,0.1, 0.2],[0,"","0.2"])

ylim(-0.01, 0.27)
fig.savefig(joinpath(fig_path, "wholebrain_spatialinfo_significance.pdf"), bbox_inches = "tight")